## 1 环境配置

1. 创建conda环境，指定python版本为3.9

In [ ]:
conda create -n traffic_sim_dl python=3.9
conda activate traffic_sim_dl

2. 安装所需Python依赖文件

In [ ]:
pip install ipykernel
pip install hydra-core==1.1.1
pip install pytorch-lightning==1.5.10
pip install transforms3d==0.3.1
pip install opencv-python==4.5.5.64
pip install gym==0.25.2
pip install waymo-open-dataset-tf-2-12-0==1.6.4

3. 安装NVidia驱动

In [ ]:
# 可通过下列如下指令查看是否安装成功
nvidia-smi

4. 安装CUDA和cuDNN

> 推荐使用 CUDA 11.3 cuDNN 8.2.1

> 安装方法参考文章：https://www.jianshu.com/p/8fbfc2e1c6a2


In [ ]:
# 查看安装的CUDA版本
nvcc -V

# 查看安装的cuDNN版本
cat /usr/local/cuda/include/cudnn_version.h | grep CUDNN_MAJOR -A 2

5. 安装与cuda版本匹配的pytorch

> 推荐使用 pytorch 1.11.0+cu113

> 下载地址：https://pytorch.org/get-started/previous-versions/

In [ ]:
# PIP安装
pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113

# CONDA安装（二选一）
conda install pytorch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0 cudatoolkit=11.3 -c pytorch

In [2]:
import tensorflow as tf
print("------------------TensorFlow------------------")
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

import torch
print("--------------------Pytorch--------------------")
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("使用的设备：", torch.cuda.get_device_name(0))  # 获取第一个 CUDA 设备的名称
else:
    device = torch.device("cpu")
    print("CUDA 不可用，将使用 CPU。")

print("使用的设备类型：", device)
print("PyTorch 版本：", torch.__version__)
print("CUDA 版本：", torch.version.cuda)

------------------TensorFlow------------------
Num GPUs Available:  1
--------------------Pytorch--------------------
使用的设备： NVIDIA RTX 3500 Ada Generation Laptop GPU
使用的设备类型： cuda
PyTorch 版本： 1.11.0
CUDA 版本： 11.3


6. 注册Weights And Biases账号，安装本地wandb库并登录

> 网址：https://wandb.ai/home


In [ ]:
# 安装wandb库
pip install wandb==0.15.12

# 登录wandb账号
wandb login

## 2 模型使用

### 2.1 修改模型配置

1. 修改数据集所在文件夹路径

    打开`config/datamodule/h5_womd.yaml`文件并修改`data_dir`字段指向数据文件夹

2. 修改Weights And Biases日志记录信息

    打开`config/loggers/wandb.yaml`文件并修改相应字段

    + project: 项目名称，需要在wandb网站创建
    + entity: 用户名
    + group: 运行所在组的名称（自定义）
    + job_type: 运行的任务类型（训练、验证、测试）
    + name: 该次运行任务名称（自定义）
    + notes: 该次运行任务说明（自定义）

### 2.2 进行模型训练

#### 2.2.1 修改模型参数

Batch（批次）是一次性送入模型进行训练或推理的数据样本集合。在深度学习中，数据通常分成若干个批次，以便于高效处理和利用计算资源。

+ 大小：批次的大小（batch_size）指每个批次中包含的数据样本数。
+ 处理：每个批次数据都会经过前向传播、计算损失、反向传播以及更新模型参数。

Step（步）通常指一次参数更新的过程。在每个步中，模型会处理一个批次的数据，进行一次前向传播和反向传播，并更新模型参数。

+ 训练步：在训练过程中，每处理一个批次数据并更新一次模型参数称为一个训练步（training step）。
+ 验证步：在验证过程中，每处理一个批次数据并计算一次验证损失或指标称为一个验证步（validation step）。

Epoch（周期）是指模型完成对整个训练数据集的一次完整遍历。在一个 epoch 中，模型会依次处理所有的批次，直到所有数据样本都被使用一次。

+ 循环：在一个 epoch 中，模型会按顺序处理所有批次的数据，通常在开始新的一轮（epoch）之前对数据进行打乱（shuffle）。
+ 次数：训练通常需要多个 epoch，以便模型逐步优化参数并提高性能。

三者联系：

+ Batch 是数据处理的基本单位，每个 batch 包含多个样本。
+ Step 是训练过程中模型参数更新的基本单位，每个 step 处理一个 batch 并更新一次模型参数。（可通过`accumulate_grad_batches`参数调节参数更新频率）
+ Epoch 是训练过程中对整个数据集的一次完整遍历。

1. 修改batch_size

    > 参数位置：`config/datamodule/h5_womd.yaml:batch_size`


    由于模型喂入的数据量较大，显卡的显存是本模型选用batch_size大小的主要考虑因素。

    目前在batch中每添加一组数据会占用**约3GB的显存空间**，因此对于本地24GB显存显卡选用的batch_size值为8。

    batch_size选取值大小的影响：
    + 较小的batch size：通常会使模型参数更新更加频繁，每次更新会带有更多的随机性，这可以帮助模型逃脱局部最优解，但可能会导致训练过程更加不稳定。
    + 较大的batch size：可以使模型参数更新更加稳定，但每次更新的随机性较低，可能导致收敛到局部最优解。此外，较大的batch size会减少每个epoch中的批次数，从而加快单个epoch的训练时间。



2. 修改Adam优化器的学习率

    + Adam优化器是什么？

        Adam（Adaptive Moment Estimation）优化器是一种广泛使用的自适应学习率优化算法，它结合了动量方法和RMSProp方法的优点，通过计算梯度的一阶矩和二阶矩的移动平均值来调整学习率。它为每个参数计算自适应的学习率，使得每个参数能够根据其梯度的变化幅度调整学习率。这在梯度不同尺度的情况下尤其有用，可以避免一些参数更新过快或过慢。

    + Adam优化器的全局学习率

        > 参数位置：`config/model/traffic_bots.yaml:optimizer.lr`

        全局学习率是Adam优化器的一个初始超参数，通常用lr表示。这个全局学习率用于整体缩放每个参数的更新步长，调整各参数自适应学习率的大小。

        原论文中使用的初始学习率为`3e-4`，参考线性缩放法则（当批量大小增加k倍时，将学习率也增大k倍），原论文中的batch大小为4*6=24，而本机训练时的批大小为8，因此将**初始学习率相应缩小三倍修改为`1e-4`**

    + Adam优化器中学习率大小对训练效果的影响

        1. 学习率过大

            梯度震荡和不稳定：过大的学习率会导致参数更新幅度过大，从而引起梯度震荡，使得损失函数不稳定，甚至可能导致训练过程发散。

            跳过最优点：学习率过大会使得优化过程跳过损失函数的最优点，无法收敛到全局最优或局部最优解。

        2. 学习率过小

            训练速度慢：过小的学习率会使参数更新幅度过小，导致模型需要更多的迭代次数才能达到收敛。这会显著增加训练时间。

            局部最优陷阱：虽然小学习率有助于精细调整参数，但也可能会陷入局部最优，难以跳出较差的局部最优点，无法有效地探索整个损失函数空间。

        3. 适当的学习率

            平衡更新：适当的学习率可以在训练速度和稳定性之间取得平衡，确保参数更新幅度合适，既能快速降低损失函数值，又能稳定收敛。
            
            有效探索：合适的学习率使模型能够有效地探索损失函数空间，从而找到全局最优或更好的局部最优解。

    + 设置阶梯下降学习率调整策略

        > 参数位置：`config/model/traffic_bots.yaml:lr_scheduler.gamma/step_size`

        随着训练的epoch推进逐渐减小全局学习率。参考论文中的超参数选取梯度下降间隔为7epoch，单次下降幅度为0.5。

        调整策略：

        1. 初始实验：选择合理的初始值，如step size为10，gamma为0.1。

        2. 监控性能：在训练过程中，关注训练损失和验证损失的变化。如果验证损失在下降间隔后没有显著改善，可能需要调整step size或gamma。

        3. 逐步调整：根据性能变化逐步调整参数，例如，如果发现学习率下降过快导致训练过早停止，可以增大step size或减小gamma。







#### 2.2.2 修改训练参数

> 参数位置：`config/trainer/default.yaml`

In [ ]:
trainer = pytorch_lightning.Trainer(
    limit_train_batches=0.2,        # 每个epoch只使用20%的训练数据
    limit_val_batches=50,           # 每个epoch只使用50个epoch的验证数据
    limit_test_batches=1.0,         # 测试时使用所有的测试数据
    num_sanity_val_steps=1,         # 设置在训练开始前运行验证集的batch数，用于检查训练代码和数据是否正常。设置为0时跳过该检查。
    max_epochs=None,                # 没有设定最大epoch数
    min_epochs=None,                # 没有设定最小epoch数
    log_every_n_steps=200,          # 每200个训练step记录一次日志
    gradient_clip_val=5,            # 梯度裁剪值为5（防止梯度爆炸）
    track_grad_norm=2,              # 追踪L2范数的梯度
    gpus=-1,                        # 使用所有可用的GPU
    precision=16,                   # 使用半精度浮点数(FP16)训练
    benchmark=False,                # 不启用benchmark
    deterministic=False,            # 不要求确定性
    sync_batchnorm=False,           # 不同步批量归一化
    detect_anomaly=False,           # 不检测训练过程中的异常
    accumulate_grad_batches=1,      # 不进行梯度累积（设为2时，每两个batch计算一次梯度）
    resume_from_checkpoint=None,    # 不从检查点恢复训练
    enable_progress_bar=True        # 启用进度条
)

+ 半精度（FP16）可以在确保数值稳定性的前提下，显著减少存储需求和提高计算速度。在处理大规模数据集或模型、需要优化训练速度和硬件资源使用的情况下，可以考虑使用半精度。

#### 2.2.3 运行训练

1. 修改`config/run.yaml`中`action`字段为`fit`，将模型指定到训练状态

2. 运行`src/run.py`

In [3]:
import os 
import sys
sys.path.append(os.path.join(os.getcwd(), 'src'))

import hydra
from omegaconf import OmegaConf

import hydra
import torch
from omegaconf import DictConfig, OmegaConf
from typing import List
from pytorch_lightning import seed_everything, LightningDataModule, LightningModule, Trainer, Callback
from pytorch_lightning.loggers import LightningLoggerBase

In [4]:
# 加载配置信息到变量
hydra.initialize(config_path="configs/")
config = hydra.compose(config_name="run.yaml")

# 输出配置信息
print(OmegaConf.to_yaml(config))

work_dir: ${hydra:runtime.cwd}
seed: 2023
action: fit
trainer:
  _target_: pytorch_lightning.Trainer
  limit_train_batches: 0.5
  limit_val_batches: 0.5
  limit_test_batches: 1.0
  num_sanity_val_steps: 0
  max_epochs: null
  min_epochs: null
  log_every_n_steps: 200
  gradient_clip_val: 5
  track_grad_norm: 2
  gpus: -1
  precision: 16
  benchmark: false
  deterministic: false
  sync_batchnorm: false
  detect_anomaly: false
  accumulate_grad_batches: 1
  resume_from_checkpoint: null
  enable_progress_bar: true
model:
  _target_: pl_modules.waymo_motion.WaymoMotion
  time_step_current: 10
  time_step_gt: 90
  time_step_end: 90
  time_step_sim_start: 1
  hidden_dim: 128
  n_video_batch: 3
  n_joint_future: 6
  interactive_challenge: false
  pre_processing:
    scene_centric:
      _target_: data_modules.scene_centric.SceneCentricPreProcessing
    input:
      _target_: data_modules.sc_input.SceneCentricInput
      dropout_p_history: -1
      pe_dim: 96
      pose_pe:
        map: pe_xy_

In [5]:
# 设置随机数种子
seed_everything(config.seed, workers=True)

# 实例化DataModule: 组织和管理数据加载
datamodule: LightningDataModule = hydra.utils.instantiate(config.datamodule)

Global seed set to 2023


In [6]:
# 实例化回调函数
callbacks: List[Callback] = []
if "callbacks" in config:
    for _, cb_conf in config.callbacks.items():
        callbacks.append(hydra.utils.instantiate(cb_conf))

# 实例化日志记录器
loggers: List[LightningLoggerBase] = []
if "loggers" in config:
    for _, lg_conf in config.loggers.items():
        loggers.append(hydra.utils.instantiate(lg_conf))

In [7]:
# 实例化模型
model: LightningModule = hydra.utils.instantiate(
    config.model, data_size=datamodule.tensor_size_train, _recursive_=False
)

In [8]:
# 使用训练器进行训练
trainer: Trainer = hydra.utils.instantiate(
    config.trainer, callbacks=callbacks, logger=loggers, _convert_="partial"
)
trainer.fit(model=model, datamodule=datamodule)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yangyh408 (use `wandb login --relogin` to force relogin)
2024-07-12 14:02:25.075197: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



   | Name                           | Type                | Params
------------------------------------------------------------------------
0  | pre_processing                 | Sequential          | 0     
1  | model                          | TrafficBots         | 3.3 M 
2  | action_head                    | ActionHead          | 50.3 K
3  | train_metrics_train            | TrainingMetrics     | 0     
4  | train_metrics_reactive_replay  | TrainingMetrics     | 0     
5  | err_metrics_reactive_replay    | ErrorMetrics        | 0     
6  | rule_metrics_reactive_replay   | TrafficRuleMetrics  | 0     
7  | waymo_post_processing          | WaymoPostProcessing | 0     
8  | womd_metrics_reactive_replay   | WOMDMetrics         | 0     
9  | err_metrics_joint_future_pred  | ErrorMetrics        | 0     
10 | rule_metrics_joint_future_pred | TrafficRuleMetrics  | 0     
11 | womd_metrics_joint_future_pred | WOMDMetrics         | 0     
12 | wosac_post_processing          | WOSACPostProcessi

Epoch 0:   0%|          | 85/61222 [02:43<32:41:31,  1.93s/it, loss=6.72, v_num=wixg]

/home/gz0779601/miniconda3/envs/traffic_bots/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


### 2.3 进行模型推理

#### 2.3.1 下载训练后的模型

> 模型保存在Weights And Biases对应项目中的Artifacts目录下

In [ ]:
import os
from pytorch_lightning.loggers import WandbLogger

# 下载wandb记录的checkpoint
def download_checkpoint(loggers, wb_ckpt) -> None:
    print(f"=> downloading checkpoint {wb_ckpt} into 'ckpt/model.ckpt'")
    if os.environ.get("LOCAL_RANK", 0) == 0:
        artifact = loggers[0].experiment.use_artifact(wb_ckpt, type="model")
        artifact_dir = artifact.download("ckpt")

loggers = [WandbLogger(
  project="traffic_bots",
  entity="yangyh408",
  name="split inference",
  notes="jupyter single run"
)]

# checkpoint_version = 'yangyh408/traffic_bots/285yb3yb:v59'
checkpoint_version = 'yangyh408/traffic_bots/2cti1q5z:v41'
download_checkpoint(loggers, checkpoint_version)

wandb: Network error (TransientError), entering retry loop.


#### 2.3.2 加载模型

通过 `load_from_checkpoint` 方法，PyTorch Lightning 提供了一种方便的方法来管理和恢复模型训练过程，使得训练中断恢复和模型部署变得更加简单和高效。

> 一般配合训练器 `trainer` 的 `save_checkpoint` 方法一起使用

使用 load_from_checkpoint 的主要效果包括：

+ 恢复模型权重：模型的权重会被恢复到保存检查点时的状态。
+ 恢复优化器状态（如果保存）：如果在保存检查点时包含了优化器状态，则可以恢复优化器的状态，使得你可以继续训练而不会失去之前的优化进展。
+ 恢复训练状态（如果保存）：可以恢复 epoch、batch 等训练状态，继续训练时从中断的地方开始。
+ 推理：加载的模型可以用于推理，从而在新的数据上进行预测。

In [10]:
# 修改模型的学习率
# ckpt_path = "ckpt/model.ckpt"
# checkpoint = torch.load(ckpt_path)
# checkpoint['optimizer_states'][0]['param_groups'][0]['lr'] = 3e-4
# checkpoint['optimizer_states'][0]['param_groups'][1]['lr'] = 3e-4
# torch.save(checkpoint, "ckpt/model_modified.ckpt")

import os 
import sys

local_module_path = os.path.join(os.getcwd(), 'src')
if local_module_path not in sys.path:
    sys.path.append(local_module_path)

from pl_modules.waymo_motion import WaymoMotion

# 加载修改后的模型
CKPG_NAME = "2cti1q5z"
CKPG_VER = "v41"

ckpt_path = f"checkpoints/{CKPG_NAME}_{CKPG_VER}.ckpt"
checkpoint_version = f"yangyh408/traffic_bots/{CKPG_NAME}:{CKPG_VER}"

model = WaymoMotion.load_from_checkpoint(ckpt_path, wb_artifact=checkpoint_version)

#### 2.3.3 运行单批次推理

In [11]:
# 加载数据集

from data_modules.data_h5_womd import DataH5womd
DATA_DIR = "/home/gz0779601/codes/DataDriven_BehaviourModels/deep_learning_model/v1.0/data/h5_womd_sim_agent"
# DATA_DIR = "/media/yangyh408/4A259082626F01B9/h5_womd_sim_agent"

single_data = DataH5womd(
    data_dir = DATA_DIR,
    filename_train = "training",
    filename_val = "validation",
    filename_test = "testing",
    n_agent = 64,
    batch_size = 4,
    num_workers = 4,
)
single_data.setup(stage="validate")
val_dataloader = single_data.val_dataloader()

In [13]:
import torch

target_batch_idx = 3
for batch_idx, batch in enumerate(val_dataloader):
    if target_batch_idx == None or batch_idx == target_batch_idx:
        model.eval()
        with torch.no_grad():
            rollout = model.manual_reactive_replay(batch=batch, batch_idx=batch_idx, log_video=False)
        break

## 3 训练架构

### 3.1 pytorch-lightning框架

+ 框架内各模块的功能

    1. DataModule

        DataModule 主要负责数据加载和预处理，将数据准备好以供模型训练使用。它将数据集的准备、划分（如训练集、验证集、测试集）、数据加载器（DataLoader）的定义等任务进行封装，使得在训练过程中可以轻松地管理和调用数据。

    2. Model

        Model 是定义和组织深度学习模型及其训练过程的核心。它包含了模型的结构（网络架构）、前向传播逻辑、损失函数定义等。

    3. Logger

        Logger 用于记录和存储训练过程中的指标、损失、学习率等信息，以便后续分析和可视化。

    4. Callbacks

        Callbacks 提供了一种机制，允许在训练过程中插入自定义的操作、逻辑或者修改训练行为。常见的用途包括记录指标、模型检查点、动态调整学习率等。

        本模型使用了三种回调函数：

        + `ModelCheckpointWB`: 用于向wandb传输最优的checkpoints信息
        + `LearningRateMonitor`: 用于监控和记录训练过程中学习率的变化情况
        + `StochasticWeightAveraging`: 用于在训练过程中实现随机权重平均（SWA）。SWA 是一种在训练结束时对模型参数进行平均的技术，旨在提升模型的泛化能力和鲁棒性。


    5. Resume

        Resume 指的是从之前保存的检查点恢复训练的操作。它允许在中断的训练任务上继续进行，而不必从头开始重新训练。

+ `trainer`训练器的调度流程

    1. 初始化 Trainer

        当你 Trainer 对象时，它会按照预设的参数配置自动初始化，同时也会初始化相关的模块。例如，初始化 Logger、注册 Callbacks 等。

    2. 准备数据（DataModule）

        在 Trainer 开始训练之前，会先调用 DataModule 的准备数据阶段。这包括调用 prepare_data() 方法来下载、预处理数据，然后调用 setup() 方法来设置数据集和数据加载器。

        + `DataLoader` 是 PyTorch 的基础组件，负责数据的批次加载和处理。

        + `DataModule` 是 PyTorch Lightning 的高级组件，用于更系统化地处理数据，包含数据下载、预处理和定义不同阶段的数据加载器（训练、验证、测试）。

    3. 初始化模型（Model）

        一旦数据准备完成后，Trainer 开始初始化 Model。这包括实例化模型对象、配置优化器和损失函数等。模型的初始化通常是在数据准备完成后，确保可以正确加载数据并进行训练。

    4. 训练循环

        一旦模型和数据都准备就绪，Trainer 开始执行训练循环。在训练循环中，它会按照以下步骤调度模块的使用：

        + 每个 Epoch 开始：
        
            在每个 epoch 开始时，Trainer 会依次调用注册的 Callbacks 的 on_epoch_start 方法，执行一些特定于 epoch 的操作，如动态调整学习率、记录 epoch 开始时间等。
        
        + Batch 训练：
        
            每次迭代中，Trainer 会从 DataModule 中获取一个 batch 的数据，并将其送入 Model 进行前向传播、损失计算、反向传播和参数更新。
        
        + Logging 和 Callbacks：
            
            在每个训练步骤结束时，Trainer 会更新指标（如损失、准确率等）并将其记录到 Logger 中。同时，会调用注册的 Callbacks 的相应方法，例如 on_batch_end，允许用户执行自定义操作。
        
        + 每个 Epoch 结束：
        
            当一个 epoch 结束时，Trainer 会调用 DataModule 的 train_dataloader() 来获取下一个 epoch 的训练数据。然后，会调用注册的 Callbacks 的 on_epoch_end 方法执行一些 epoch 结束时的操作，如模型验证、保存检查点等。
    
    5. 验证和测试
    
        在训练的过程中，Trainer 还会根据配置周期性地执行验证和测试步骤。它会使用 DataModule 提供的验证和测试数据集，并调用 Model 进行推理或评估，同时记录和更新指标。

    6. 检查点和恢复
        
        如果中断了训练或需要从之前的检查点恢复训练，Trainer 在初始化时会检查是否有指定的 resume_from_checkpoint 参数，并在需要时加载模型和优化器的状态，从中断的地方继续训练。